In [38]:
"""
Here the particles are made to converge so as to find a sequence
which is able to achieve the target score
"""
import random
import sys
import numpy as np
import copy
import pickle
import json
import os.path

In [5]:
TARGET = 50
MAX_INPUTS = 10
MAX_PARTICLES = 10
V_MAX = 10 #Maximum velocity change allowed
MAX_EPOCHS = 200
START_RANGE_MIN = 140
START_RANGE_MAX = 200

In [6]:
class Particle:
    def __init__(self, pBest):
        self.data = [0] *  MAX_INPUTS
        self.pBest = pBest
        self.velocity = 0.0
    
    def get_data(self):
        return self.data
    
    def set_data(self, value):
        self.data = value
    
    def set_pBest(self, value):
        self.pBest = value
        
    def get_pBest(self):
        return self.pBest
    
    def get_velocity(self):
        return self.velocity
    
    def set_velocity(self, value):
        self.velocity = value

In [7]:
def initialize_Particles():
    particles = []
    for i in range(MAX_PARTICLES):
        newParticle = Particle(0)
        total = 0
        data = []
        for j in range(MAX_INPUTS):
            data.append(np.floor(np.random.uniform(START_RANGE_MIN, START_RANGE_MAX)))
        
        newParticle.set_data(data)
        newParticle.set_pBest(np.sum(data))
        particles.append(copy.deepcopy(newParticle))
    
    return particles

In [8]:
def test_problem(index, particles):
    val = 0
    a = particles[index].get_data()
    for i in a:
        val += i
    return val

In [9]:
def max_fitness(particles):
    minimum = 0
    for i in range(MAX_PARTICLES):
        #print("i: " + str(i))
        if i!=minimum: 
            if np.abs(TARGET - test_problem(i, particles))<np.abs(TARGET - test_problem(minimum, particles)):
                minimum = i
    
    return minimum

In [10]:
def calc_velocity(gBestIndex, particles):
    
    bestVal = test_problem(gBestIndex, particles)
    for i in range(MAX_PARTICLES):
        testVal = test_problem(i, particles)
        velocity = particles[i].get_velocity() + (2 * np.random.uniform() * (bestVal - testVal))
        velocity += (2 * np.random.uniform() * (particles[i].get_pBest() - testVal))
        
        if velocity > V_MAX:
            velocity = V_MAX
        elif velocity < -V_MAX:
            velocity = -V_MAX
        
        particles[i].set_velocity(velocity)

In [11]:
def update_particles(particles, gBest):
    for i in range(MAX_PARTICLES):
        data = particles[i].get_data()
        temp = particles[gBest].get_data()
        vel = particles[i].get_velocity()
        for y in range(len(data)):
            if data[y] != temp[y]:
                data[y] += vel
                data[y]=np.floor(data[y])
        particles[i].set_data(data)
        
        total = test_problem(i, particles)
        if np.abs(TARGET - total) < particles[i].get_pBest():
            particles[i].set_pBest(total)

In [82]:
def PSO_algo():
    epoch = 1
    result = []
    solution = -1
    
    particles = initialize_Particles()
    random.seed(10)
    
    while epoch<MAX_EPOCHS:

        gBest = max_fitness(particles)
        
        save_data = {}
        save_data['epoch'] = epoch
        save_data['gBest'] = gBest
        with open('save.json', 'w') as fp:
            json.dump(save_data, fp, sort_keys=True, indent=4)

        for i in range(MAX_PARTICLES):
            if test_problem(i, particles)==TARGET:
                result = particles[i].get_data()
                solution = i
                break
        
        if solution>=0:
            print(result)
            break
                
        calc_velocity(gBest, particles)
        update_particles(particles, gBest)
        
        #print("Best in epoch " + str(epoch) +": " + str(particles[gBest].get_data()))
        print(str(np.sum(particles[gBest].get_data())))
        with open('data.pk1', 'w') as fp:
            pickle.dump(particles, fp, pickle.HIGHEST_PROTOCOL)
        
        epoch += 1
    print(particles[gBest].get_data()[3])
    

In [87]:
#print(max_fitness())
PSO_algo()

1674.0
1593.0
1507.0
1420.0
1328.0
1235.0
1138.0
1074.0
999.0
901.0
823.0
726.0
637.0
540.0
448.0
355.0
288.0
194.0
129.0
[16.0, 3.0, 1.0, -27.0, 17.0, 16.0, 25.0, -20.0, 5.0, 14.0]
-27.0


In [85]:
with open('data.pk1', 'rb') as input:
    par1 = pickle.load(input)
    for i in range(len(par1)):
        print(par1[i].get_data(), np.sum(par1[i].get_data()))

([26.0, -26.0, 21.0, 9.0, 3.0, 22.0, -3.0, -32.0, -23.0, 18.0], 15.0)
([1.0, 27.0, -13.0, 9.0, -15.0, 27.0, -17.0, -18.0, 36.0, 3.0], 40.0)
([1.0, 27.0, 21.0, 9.0, -5.0, -3.0, -15.0, -11.0, 7.0, 18.0], 49.0)
([-13.0, 22.0, -8.0, -3.0, -23.0, -4.0, -13.0, 50.0, 15.0, 3.0], 26.0)
([4.0, -34.0, 21.0, 9.0, -29.0, 22.0, -11.0, 1.0, 17.0, -25.0], -25.0)
([-11.0, 24.0, 21.0, -14.0, -5.0, 36.0, -15.0, -11.0, 7.0, 18.0], 50.0)
([-52.0, 4.0, 21.0, -16.0, -5.0, 18.0, -12.0, -1.0, 7.0, -8.0], -44.0)
([21.0, 9.0, 21.0, 3.0, 8.0, -14.0, -15.0, 16.0, 7.0, 25.0], 81.0)
([-14.0, 21.0, -20.0, 17.0, -28.0, -3.0, 5.0, 63.0, 42.0, -17.0], 66.0)
([15.0, -9.0, -2.0, -22.0, 27.0, -3.0, 18.0, 54.0, -21.0, 4.0], 61.0)


In [77]:
if(os.path.isfile('data.pk1')):
    print("found")
    
with open('save.json', 'r') as ip:
    open_data = json.load(ip)

print(open_data['gBest'])

found
2
